In [1]:
import os
import keras
import numpy as np
import sagemaker

Using TensorFlow backend.


In [5]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip
os.makedirs("./data", exist_ok = True)
!mv fra.txt data/train.txt

--2020-07-10 04:37:10--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.24.108.196, 104.24.109.196, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6041598 (5.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   5.76M  4.78MB/s    in 1.2s    

2020-07-10 04:37:12 (4.78 MB/s) - ‘fra-eng.zip’ saved [6041598/6041598]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [3]:
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='data/handson-byom-tensorflow-keras')
print('Input data is uploaded to: {}'.format(input_data))

Input data is uploaded to: s3://sagemaker-ap-northeast-1-889905360474/data/handson-byom-tensorflow-keras


In [4]:
!wget https://raw.githubusercontent.com/keras-team/keras/master/examples/lstm_seq2seq.py .

--2020-07-10 04:36:09--  https://raw.githubusercontent.com/keras-team/keras/master/examples/lstm_seq2seq.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9348 (9.1K) [text/plain]
Saving to: ‘lstm_seq2seq.py’

lstm_seq2seq.py     100%[===================>]   9.13K  --.-KB/s    in 0s      

2020-07-10 04:36:09 (70.9 MB/s) - ‘lstm_seq2seq.py’ saved [9348/9348]

--2020-07-10 04:36:09--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2020-07-10 04:36:09--
Total wall clock time: 0.3s
Downloaded: 1 files, 9.1K in 0s (70.9 MB/s)


In [6]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
train_instance_type = "ml.c5.xlarge"
role = get_execution_role()
estimator = TensorFlow(entry_point = "./lstm_seq2seq.py",
                       role=role,
                       train_instance_count=1,
                       train_instance_type=train_instance_type,
                       framework_version="1.12.0",
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'batch-size': 64,
                                        'num-classes': 10,
                                        'epochs': 100})
estimator.fit(input_data)

2020-07-10 04:39:15 Starting - Starting the training job...
2020-07-10 04:39:17 Starting - Launching requested ML instances.........
2020-07-10 04:41:01 Starting - Preparing the instances for training...
2020-07-10 04:41:39 Downloading - Downloading input data...
2020-07-10 04:42:06 Training - Training image download completed. Training in progress..2020-07-10 04:42:08,487 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-07-10 04:42:08,492 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-10 04:42:08,754 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-10 04:42:08,768 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-10 04:42:08,777 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "curren

In [7]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
endpoint = predictor.endpoint

-------------!

In [8]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import tensorflow as tf
from keras import backend as K
    
batch_size = 64  # Batch size for training.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'data/train.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [9]:
from sagemaker.tensorflow.serving import Predictor

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    encoder_predictor = Predictor(endpoint, model_name='model1')
    states_value = (encoder_predictor.predict(input_seq))['predictions'][0]
    h = states_value['lstm_1/while/Exit_2:0']
    c = states_value['lstm_1/while/Exit_3:0']

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, num_decoder_tokens))#np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    decoder_predictor = Predictor(endpoint, model_name='model2')
    while not stop_condition:
        #output_tokens, h, c 
        decoder_result= (decoder_predictor.predict(
            {"instances":[{ 'input_2:0':target_seq.tolist(), 'input_3:0':h, 'input_4:0':c}]}))['predictions'][0]
        
        output_tokens = decoder_result['dense_1_1/truediv:0']
        h = decoder_result['lstm_2_1/while/Exit_2:0']
        c = decoder_result['lstm_2_1/while/Exit_3:0']

        # Sample a token
        sampled_token_index = np.argmax(output_tokens)#output_tokens[0, -1, :]
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, num_decoder_tokens))
        target_seq[0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [10]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decode_sequence(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Je                                                          
